<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/multi_stock_ppo_live_inference_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [ ]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,775 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,168 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,207

In [ ]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [ ]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [ ]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.0/405.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.2/851.2 kB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.7/557.7 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

In [ ]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


  Using cached gym_anytrading-2.0.0-py3-none-any.whl.metadata (292 bytes)
  Using cached stable_baselines3-2.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached gym_anytrading-2.0.0-py3-none-any.whl (172 kB)
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (13.8 MB)
Using cached nvidia_cuda_nvrtc_cu1

In [ ]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0


import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


In [ ]:
# Install all required packages
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance --quiet
!pip install stable-baselines3[extra] --quiet


In [ ]:
!pip install transformers pandas scikit-learn torch torchvision torchaudio matplotlib seaborn


In [ ]:
import os
import pandas as pd
import torch
import gc
from transformers import pipeline


In [ ]:
import os
import gc
import pandas as pd
import torch
from transformers import pipeline

# Load FinBERT sentiment model
sentiment_model = pipeline("sentiment-analysis", model="ProsusAI/finbert")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:

#Step 7: authenticate with hugging face hub (optional)
#This allows for better access and avoids rate limits when downloading public models/datasets

# Authenticate with Hugging Face Hub
#notebook_login()


In [ ]:
#  Import Required Libraries
import gc
import json
import os
import random
import time
from collections import deque
from datetime import datetime, timedelta

import cupy as cp
import cudf
import cuml
import dask
import gymnasium as gym
import gym_anytrading
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import torch
import xgboost as xgb
import yfinance as yf
from cuml.ensemble import RandomForestClassifier
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv

#  Ticker List and CONFIG
ticker_list = [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

strategy_name = "ppo_walkforward_v1"

CONFIG = {
    'symbols': ticker_list,
    'period': '720d',
    'interval': '1h',
    'target': 'Target',
    'sharpe_threshold': 1.5,
    'return_threshold': 1.25,
    'strategy_name': strategy_name
}

#  Download Function with Date Logic
def download_stock_data(ticker, start_date=None, end_date=None, interval="1h", max_retries=5, window_days=730):
    if start_date is None or end_date is None:
        end_date = datetime.today()
        start_date = end_date - timedelta(days=window_days)

    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')

    for attempt in range(1, max_retries + 1):
        try:
            print(f"Attempt {attempt}: Downloading {ticker} from {start_str} to {end_str}...")
            df = yf.download(ticker, start=start_str, end=end_str, interval=interval, progress=False)
            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                df['Datetime'] = pd.to_datetime(df['Datetime'] if 'Datetime' in df.columns else df['Date'])
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying in {attempt * 5} sec...")
            time.sleep(attempt * 5)

    print(f" Failed to download {ticker}")
    return None

#  Feature Engineering Function
def compute_enhanced_features(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100
    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    df['CCI'] = ((df['High'] + df['Low'] + df['Close']) / 3 - df['Close'].rolling(20).mean()) / (0.015 * df['Close'].rolling(20).std())
    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['MACD_Line'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()
    df['RSI'] = 100 - (100 / (1 + df['Close'].diff().apply(lambda x: x if x > 0 else 0).rolling(14).mean() /
                              -df['Close'].diff().apply(lambda x: x if x < 0 else 0).rolling(14).mean()))

    #  True Range and ATR (Fixed)
    tr = pd.concat([
        df['High'] - df['Low'],
        abs(df['High'] - df['Close'].shift()),
        abs(df['Low'] - df['Close'].shift())
    ], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(14).mean()

    df['Volatility'] = df['Close'].pct_change().rolling(20).std()
    df['Return'] = (df['Close'].shift(-10) - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)
    df.dropna(inplace=True)
    return df

#  Dynamic Walkforward Feature Engineering Pipeline
all_dfs = []
end_date = datetime.today()
start_date = end_date - timedelta(days=729)  # Keep within Yahoo's limit

for ticker in CONFIG['symbols']:
    df_single = download_stock_data(
        ticker=ticker,
        start_date=start_date,
        end_date=end_date,
        interval="1h"
    )

    if df_single is not None and not df_single.empty:
        try:
            df_features = compute_enhanced_features(df_single)
            all_dfs.append(df_features)
        except Exception as e:
            print(f" Feature engineering failed for {ticker}: {e}")
    else:
        print(f" No data for {ticker}")


#  Combine and Save
if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f" Combined dataset created with shape: {df.shape}")
    df.to_csv("multi_stock_feature_engineered_dataset.csv", index=False)
    print(" Saved locally to multi_stock_feature_engineered_dataset.csv")

    drive_path = "/content/drive/MyDrive/trading_data/"
    os.makedirs(drive_path, exist_ok=True)
    df.to_csv(os.path.join(drive_path, "multi_stock_feature_engineered_dataset.csv"), index=False)
    print(f" Also saved to Google Drive at {drive_path}multi_stock_feature_engineered_dataset.csv")
else:
    print(" No usable data found for any ticker.")


/usr/local/lib/python3.11/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 314, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


Attempt 1: Downloading AAPL from 2023-08-03 to 2025-08-01...


/tmp/ipython-input-425754791.py:69: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_str, end=end_str, interval=interval, progress=False)


Attempt 1: Downloading TSLA from 2023-08-03 to 2025-08-01...


/tmp/ipython-input-425754791.py:69: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_str, end=end_str, interval=interval, progress=False)


...

Attempt 1: Downloading SBUX from 2023-08-03 to 2025-08-01...


/tmp/ipython-input-425754791.py:69: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_str, end=end_str, interval=interval, progress=False)


Attempt 1: Downloading ORCL from 2023-08-03 to 2025-08-01...


/tmp/ipython-input-425754791.py:69: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_str, end=end_str, interval=interval, progress=False)


 Combined dataset created with shape: (182849, 26)
 Saved locally to multi_stock_feature_engineered_dataset.csv
 Also saved to Google Drive at /content/drive/MyDrive/trading_data/multi_stock_feature_engineered_dataset.csv


In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!rm -rf /content/drive

In [ ]:
# === PPO Walkforward Strategy with Execution, Risk, Filtering, Regime, and Tuning Support (Fixed) ===
import os, gc, time, json, logging, torch
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box
from transformers import pipeline

# === Config ===
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/ppo_walkforward_results"
FINAL_MODEL_DIR = os.path.join(RESULTS_DIR, "models")
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

test_mode = True
simulate_latency = True

# === Logging ===
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")

# === Load Data ===
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
df['Datetime'] = pd.to_datetime(df['Datetime'])
df['SentimentScore'] = df.get('SentimentScore', 0)
sentiment_model = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# === Custom PPO Environment ===
class ContinuousTradingEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size):
        super().__init__(df=df.reset_index(drop=True), frame_bound=frame_bound, window_size=window_size)
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def step(self, action):
        try:
            discrete_action = 0 if action < -0.3 else 1 if action > 0.1 else 2
            result = super().step(discrete_action)
            obs_raw, reward, terminated, truncated, info = result if len(result) == 5 else (*result, False)
            obs = np.array(obs_raw[1], dtype=np.float32).flatten()

            i = self._current_tick
            price_now = self.df.loc[i, 'Denoised_Close']
            price_prev = self.df.loc[i - 1, 'Denoised_Close']
            volatility = self.df.loc[i, 'Volatility']
            sentiment = self.df.loc[i, 'SentimentScore']
            mean = self.df['Denoised_Close'].rolling(20).mean().iloc[i]
            std = self.df['STD_20'].iloc[i]
            momentum = price_now - self.df['Denoised_Close'].iloc[max(0, i - 10)]

            action_val = float(action.flatten()[0])
            confidence = abs(price_now - mean) / (std + 1e-6)
            confidence_bonus = 0.1 * confidence if confidence > 1.5 else 0
            momentum_bonus = 0.2 if discrete_action == 1 and momentum > 0 else 0

            whipsaw_penalty = 0
            if hasattr(self, 'last_action_val'):
                if abs(action_val - self.last_action_val) > 1.0 and (i - self.last_action_idx) < 5:
                    whipsaw_penalty = 0.05
            self.last_action_idx, self.last_action_val = i, action_val

            reward += (price_now - price_prev) * discrete_action * np.clip(volatility / 0.01, 0.5, 2.0)
            reward += confidence_bonus + momentum_bonus
            reward += 0.05 * sentiment if discrete_action in [1, -1] else 0
            reward -= whipsaw_penalty + 0.001 * abs(action_val)

            if 'Regime' in self.df.columns:
                if self.df.loc[i, 'Regime'] == 1:
                    reward *= 0.7
                elif self.df.loc[i, 'Regime'] == 2:
                    reward *= 0.5

            return obs, np.clip(reward, -1.0, 1.0), terminated, truncated, info
        except Exception as e:
            logging.error(f"Step error: {e}")
            result = self.reset()
            obs_raw = result if isinstance(result, tuple) else (result, {})
            obs = np.array(obs_raw[1], dtype=np.float32).flatten()
            return obs, 0, True, False, {}

# === Window Generator ===
def get_walk_forward_windows(df, window_size=3000, step_size=500, min_len=1000):
    return [(start, start + window_size) for start in range(0, len(df) - min_len, step_size) if start + window_size < len(df)]

# === PPO Trainer ===
def walkforward_ppo(df, ticker, completed_tickers, window_size=3000, step_size=500, timesteps=1500000):
    if ticker in completed_tickers:
        logging.info(f"✅ Skipping {ticker} — already completed")
        return []

    logging.info(f"🚀 Starting {ticker}")
    results = []
    device = "cpu"
    windows = get_walk_forward_windows(df, window_size, step_size)

    for w_idx, (start, end) in enumerate(windows):
        window_start_time = time.time()
        gc.collect()

        df_window = df.iloc[start:end].reset_index(drop=True)
        avg_sentiment = df_window['SentimentScore'].mean()
        logging.info(f"{ticker} | Window {w_idx+1}: Avg Sentiment = {avg_sentiment:.2f}")

        if len(df_window) <= 52 or len(df_window) % 2 != 0:
            df_window = df_window.iloc[:-1]
            df_window = (df_window - df_window.mean()) / df_window.std()

        if 'EMA_200' in df_window.columns and not (df_window['EMA_50'].iloc[-1] > df_window['EMA_200'].iloc[-1]):
            logging.info(f"{ticker} | Window {w_idx+1}: Skipped (no uptrend)")
            continue

        if 'Regime' in df_window.columns and df_window['Regime'].iloc[-1] in [1, 2]:
            logging.info(f"{ticker} | Window {w_idx+1}: Skipped (unfavorable regime)")
            continue

        logging.info(f"{ticker} | Window {w_idx+1}/{len(windows)} | {len(df_window)} rows")
        frame_bound = (50, len(df_window) - 3)
        vec_env = DummyVecEnv([lambda: ContinuousTradingEnv(df=df_window, frame_bound=frame_bound, window_size=10)])
        env = VecNormalize(vec_env, norm_obs=True, norm_reward=True)

        model = PPO("MlpPolicy", env,
                    learning_rate=1.5e-5, n_steps=2048, batch_size=256,
                    n_epochs=10, gamma=0.99, gae_lambda=0.95,
                    clip_range=0.15, ent_coef=0.005,
                    device=device, verbose=0,
                    policy_kwargs=dict(net_arch=[128, 128]))

        model.learn(total_timesteps=timesteps)

        prefix = f"ppo_{ticker}_window{w_idx+1}"
        model.save(os.path.join(FINAL_MODEL_DIR, f"{prefix}.zip"))
        env.save(os.path.join(FINAL_MODEL_DIR, f"{prefix}_vecnorm.pkl"))
        json.dump({"features": df_window.columns.tolist()}, open(os.path.join(FINAL_MODEL_DIR, f"{prefix}_features.json"), "w"))

        obs = env.reset().copy()
        position, balance, portfolio, trade_log, buy_indices, sell_indices = 0, 100000, [], [], [], []
        entry_price = None

        for i in range(len(df_window)):
            action, _ = model.predict(obs, deterministic=True)
            step_result = env.step(action)
            obs, reward, terminated, truncated, info = step_result if len(step_result) == 5 else (*step_result, False)

            action_val = float(action.flatten()[0])
            price = df_window['Denoised_Close'].iloc[i]
            if simulate_latency:
                time.sleep(np.random.uniform(0.05, 0.15))

            slippage = price * 0.001
            if action_val > 0.5 and position == 0:
                qty = (0.1 * balance) / (price + slippage)
                position = qty
                entry_price = price
                balance -= qty * (price + slippage)
                trade_log.append("BUY")
                buy_indices.append(i)
            elif action_val < -0.5 and position > 0:
                balance += position * (price - slippage)
                position = 0
                entry_price = None
                trade_log.append("SELL")
                sell_indices.append(i)
            elif position > 0 and entry_price:
                if price <= entry_price * 0.95:
                    balance += position * (price - slippage)
                    position = 0
                    entry_price = None
                    trade_log.append("STOP_LOSS")
                    sell_indices.append(i)
                elif price >= entry_price * 1.10:
                    balance += position * (price - slippage)
                    position = 0
                    entry_price = None
                    trade_log.append("TAKE_PROFIT")
                    sell_indices.append(i)
                else:
                    trade_log.append("HOLD")
            else:
                trade_log.append("HOLD")

            portfolio.append(balance if position == 0 else balance + position * price)

        portfolio_series = pd.Series(portfolio)
        smoothed = portfolio_series.ewm(span=10).mean()

        final_value = portfolio_series.iloc[-1]
        hold_value = (100000 / df_window['Denoised_Close'].iloc[0]) * df_window['Denoised_Close'].iloc[-1]
        returns = portfolio_series.pct_change().fillna(0)
        sharpe = (returns.mean() / (returns.std() + 1e-6)) * np.sqrt(252)
        drawdown = ((portfolio_series.cummax() - portfolio_series) / portfolio_series.cummax()).max() * 100

        try:
            conf_scores = [
                abs(df_window['Denoised_Close'].iloc[j] - df_window['Denoised_Close'].rolling(20).mean().iloc[j]) /
                (df_window['STD_20'].iloc[j] + 1e-6)
                for j in range(20, len(df_window))
            ]
        except Exception as e:
            logging.warning(f"⚠️ Confidence score calc failed for {ticker} | Window {w_idx+1}: {e}")
            conf_scores = [0.5] * (len(df_window) - 20)

        confidence_counts = pd.Series(pd.cut(
            conf_scores, bins=[0, 1, 2, 3, np.inf], labels=["Low", "Medium", "High", "Extreme"]
        )).value_counts().to_dict()

        results.append({
            "Ticker": ticker,
            "Window": f"{start}-{end}",
            "PPO_Portfolio": round(final_value, 2),
            "BuyHold": round(hold_value, 2),
            "Sharpe": round(sharpe, 3),
            "Drawdown_%": round(drawdown, 2),
            "Confidence_Buckets": confidence_counts,
            "Winner": "PPO" if final_value > hold_value else "Buy & Hold"
        })

        plt.figure(figsize=(10, 4))
        plt.plot(smoothed, label="PPO Portfolio (Smoothed)")
        plt.axhline(hold_value, linestyle="--", color="red", label="Buy & Hold")
        plt.scatter(buy_indices, df_window['Denoised_Close'].iloc[buy_indices], color='green', marker='^', label='Buy')
        plt.scatter(sell_indices, df_window['Denoised_Close'].iloc[sell_indices], color='red', marker='v', label='Sell')
        plt.title(f"{ticker} PPO vs Buy & Hold")
        plt.legend(); plt.grid(True); plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, f"{prefix}_performance.png"))
        plt.close()

        logging.info(f"{ticker} | Window {w_idx+1} done | ⏱️ {round(time.time() - window_start_time, 2)} sec")
        del env, model, df_window
        gc.collect()
        torch.cuda.empty_cache()

    return results

# === Run All Tickers ===
def run_all():
    summary_path = os.path.join(RESULTS_DIR, "ppo_walkforward_summary.csv")
    completed = set()
    if os.path.exists(summary_path):
        try:
            summary_df = pd.read_csv(summary_path)
            completed = set(summary_df['Ticker'].unique())
        except: pass

    tickers = ['TSLA', 'NVDA'] if test_mode else df['Symbol'].unique().tolist()
    all_results = []

    def safe_run(t):
        df_t = df[df['Symbol'] == t]
        if df_t.empty:
            print(f"⚠️ Skipping {t}: no data in dataframe.")
            return []
        return walkforward_ppo(df_t, t, completed)

    with ThreadPoolExecutor(max_workers=min(len(tickers), os.cpu_count())) as executor:
        for result in executor.map(safe_run, tickers):
            if result: all_results.extend(result)
            pd.DataFrame(all_results).to_csv(summary_path, index=False)

    final = pd.DataFrame(all_results)
    if not final.empty:
        weights = final['Sharpe'].clip(lower=0.1).fillna(0.1)
        weighted_perf = np.average(final['PPO_Portfolio'], weights=weights)
        print("\n📊 Top 5 by Sharpe:")
        print(final.sort_values("Sharpe", ascending=False).head())
        print(f"\n📈 Weighted Portfolio Value (Sharpe-weighted): {round(weighted_perf, 2)}")

run_all()


Device set to use cpu


In [ ]:
# === PPO Walkforward Strategy with Final Fixes and Debugging ===
import os, gc, time, json, logging, torch
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from datetime import datetime
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from gymnasium.spaces import Box
from transformers import pipeline
from gym_anytrading.envs import StocksEnv

# === Config ===
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/ppo_walkforward_results"
FINAL_MODEL_DIR = os.path.join(RESULTS_DIR, "models")
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

test_mode = True
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# === Load Data ===
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
df['Datetime'] = pd.to_datetime(df['Datetime'])
if 'SentimentScore' not in df.columns:
    df['SentimentScore'] = 0

sentiment_model = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# === Plotting ===
def plot_performance(ticker, portfolio, hold_value):
    plt.figure(figsize=(10, 5))
    plt.plot(portfolio, label='PPO Portfolio')
    plt.axhline(hold_value, linestyle='--', color='gray', label='Buy & Hold')
    plt.title(f"{ticker} Portfolio vs Buy & Hold")
    plt.xlabel("Time Step")
    plt.ylabel("Portfolio Value")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, f"portfolio_{ticker}.png"))
    plt.close()

# === Custom Trading Environment ===
class ContinuousTradingEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size):
        super().__init__(df=df.reset_index(drop=True), frame_bound=frame_bound, window_size=window_size)
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def step(self, action):
        try:
            discrete_action = 0 if action < -0.01 else 1 if action > 0.01 else 2
            result = super().step(discrete_action)
            obs, reward, terminated, truncated, info = result if len(result) == 5 else (*result, False)

            price_now = self.df.loc[self._current_tick, 'Close']
            price_prev = self.df.loc[self._current_tick - 1, 'Close']
            volatility = self.df.loc[self._current_tick, 'Volatility']
            sentiment_score = self.df.loc[self._current_tick, 'SentimentScore']

            action_val = float(action.flatten()[0])
            mean = self.df['Close'].rolling(20).mean().iloc[self._current_tick]
            conf_score = abs(price_now - mean) / (self.df['STD_20'].iloc[self._current_tick] + 1e-6)
            confidence_bonus = 0.1 * conf_score if conf_score > 1.5 else 0

            momentum = price_now - self.df['Close'].iloc[max(0, self._current_tick - 10)]
            momentum_bonus = 0.2 if discrete_action == 1 and momentum > 0 else 0

            self.last_action_idx = getattr(self, "last_action_idx", self._start_tick)
            self.last_action_val = getattr(self, "last_action_val", None)
            whipsaw_penalty = 0
            if self.last_action_val is not None:
                recent_change = abs(action_val - self.last_action_val)
                if (self._current_tick - self.last_action_idx < 5) and (recent_change > 1.0):
                    whipsaw_penalty = 0.05

            self.last_action_idx = self._current_tick
            self.last_action_val = action_val

            reward += (price_now - price_prev) * discrete_action * np.clip(volatility / 0.01, 0.5, 2.0)
            reward += confidence_bonus + momentum_bonus
            reward += 0.05 * sentiment_score if discrete_action in [1, -1] else 0
            reward -= whipsaw_penalty
            reward -= 0.001 * abs(action_val) + 0.002 * int(action_val != 0)

            if 'Regime' in self.df.columns:
                if self.df.loc[self._current_tick, 'Regime'] == 1:
                    reward *= 0.7
                elif self.df.loc[self._current_tick, 'Regime'] == 2:
                    reward *= 0.5

            return obs, np.clip(reward, -1.0, 1.0), terminated, truncated, info
        except Exception as e:
            logging.error(f"Step error: {e}")
            return self.reset(), 0, True, True, {}

# === Walkforward Window Generator ===
def get_walk_forward_windows(df, window_size=3000, step_size=500, min_len=1000):
    return [(start, start + window_size) for start in range(0, len(df) - min_len, step_size) if start + window_size < len(df)]

# === Main PPO Walkforward Function ===
def walkforward_ppo(df, ticker, completed_tickers, window_size=3000, step_size=500, timesteps=1_000_000):
    if ticker in completed_tickers:
        logging.info(f"✅ Skipping {ticker} — already completed")
        return []

    logging.info(f"🚀 Starting {ticker}")
    results = []
    device = "cpu"
    windows = get_walk_forward_windows(df, window_size, step_size)

    for w_idx, (start, end) in enumerate(windows):
        logging.info(f"🪟 Window {w_idx+1}/{len(windows)} for {ticker} [{start}:{end}]")
        window_start_time = time.time()
        gc.collect()

        df_window = df.iloc[start:end].reset_index(drop=True)
        numeric_cols = df_window.select_dtypes(include=[np.number]).columns
        means = df_window[numeric_cols].mean()
        stds = df_window[numeric_cols].std().replace(0, 1e-6)
        df_window[numeric_cols] = (df_window[numeric_cols] - means) / stds
        df_window.dropna(inplace=True)

        frame_bound = (50, len(df_window) - 3)
        vec_env = DummyVecEnv([lambda: ContinuousTradingEnv(df=df_window, frame_bound=frame_bound, window_size=10)])
        env = VecNormalize(vec_env, norm_obs=True, norm_reward=True)
        obs = env.reset()

        model = PPO("MlpPolicy", env, learning_rate=5e-5, n_steps=2048, batch_size=256,
                    n_epochs=10, gamma=0.99, gae_lambda=0.95, clip_range=0.15, ent_coef=0.005,
                    device=device, verbose=0, policy_kwargs=dict(net_arch=[128, 128]))
        model.learn(total_timesteps=timesteps)

        logging.info(f"✅ Model trained | ⏱️ {round(time.time() - window_start_time, 2)} sec")
        prefix = f"ppo_{ticker}_window{w_idx+1}"
        model.save(os.path.join(FINAL_MODEL_DIR, f"{prefix}.zip"))

        position, balance, portfolio, trade_log, confidences, action_vals = 0, 100000, [], [], [], []
        for i in range(len(df_window)):
            action, _ = model.predict(obs, deterministic=True)
            obs_tensor = torch.tensor(obs, dtype=torch.float32).unsqueeze(0).to(model.device)
            with torch.no_grad():
                dist = model.policy.get_distribution(obs_tensor)
                confidence = torch.max(dist.distribution.probs).item() if hasattr(dist.distribution, "probs") else None
            confidences.append(confidence)
            action_val = float(action.flatten()[0])
            action_vals.append(action_val)

            obs, reward, terminated, truncated, info = env.step(action)

            price = df_window['Close'].iloc[i]
            if action_val > 0.05 and position == 0:
                position = balance / price
                balance = 0
                trade_log.append("BUY")
            elif action_val < -0.01 and position > 0:
                balance = position * price
                position = 0
                trade_log.append("SELL")
            else:
                trade_log.append("HOLD")
            portfolio.append(balance if balance > 0 else position * price)

        if not portfolio:
            logging.warning(f"⚠️ Empty portfolio for {ticker} | Window {w_idx+1}")
            continue

        final_value = portfolio[-1]
        hold_value = (100000 / df_window['Close'].iloc[0]) * df_window['Close'].iloc[-1]
        returns = pd.Series(portfolio).pct_change().fillna(0)
        sharpe = (returns.mean() / (returns.std() + 1e-6)) * np.sqrt(252)
        drawdown = ((pd.Series(portfolio).cummax() - pd.Series(portfolio)) / pd.Series(portfolio).cummax()).max() * 100

        results.append({
            "Ticker": ticker,
            "Window": f"{start}-{end}",
            "PPO_Portfolio": round(final_value, 2),
            "BuyHold": round(hold_value, 2),
            "Sharpe": round(sharpe, 3),
            "Drawdown_%": round(drawdown, 2),
            "Winner": "PPO" if final_value > hold_value else "Buy & Hold"
        })

        plot_performance(ticker, portfolio, hold_value)

        # NEW: Action Histogram
        plt.figure(figsize=(8, 4))
        plt.hist(action_vals, bins=20, color='skyblue', edgecolor='black')
        plt.title(f"{ticker} - Action Distribution (Window {w_idx+1})")
        plt.xlabel("Action Value")
        plt.ylabel("Frequency")
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, f"{prefix}_action_hist.png"))
        plt.close()

        del env, model, df_window, portfolio, trade_log
        gc.collect()
        torch.cuda.empty_cache()

    if not results:
        logging.warning(f"⚠️ No results saved for {ticker}")
    else:
        logging.info(f"✅ {ticker} completed with {len(results)} windows")

    return results

# === Main Execution ===
CONFIG = {"symbols": df['Symbol'].unique().tolist()}

if test_mode:
    ticker_list = ['AAPL']  # Or ['TSLA', 'NVDA'] for more
    for symbol in ticker_list:
        logging.info(f"🔁 Running {symbol} in test mode")
        logging.info(f"🧪 {symbol} rows: {df[df['Symbol'] == symbol].shape}")  # Debug: confirm rows
        results = walkforward_ppo(df[df['Symbol'] == symbol].copy(), symbol, completed_tickers=set())
        if results:
            pd.DataFrame(results).to_csv(os.path.join(RESULTS_DIR, f"summary_{symbol}.csv"), index=False)
        else:
            logging.warning(f"⚠️ No results for {symbol}")

else:
    for symbol in CONFIG['symbols']:
        logging.info(f"🔁 Running {symbol} in full mode")
        results = walkforward_ppo(df[df['Symbol'] == symbol].copy(), symbol, completed_tickers=set())
        if results:
            pd.DataFrame(results).to_csv(os.path.join(RESULTS_DIR, f"summary_{symbol}.csv"), index=False)
        else:
            logging.warning(f"⚠️ No results for {symbol}")


Device set to use cpu


KeyboardInterrupt: 